# SQL - Nashville Housing Project

This project focuses on data cleaning. 
1. Standardised Date Format
2. Populate property address data
3. Breaking out addresses into individual columns (Address, City, State)
4. Change Y and N to Yes and No in 'Sold as Vacant' field
5. Remove Duplicates

### Stage 1 - Uploading to Azure

This is different using the docker - to do this I have installed the following extension: SQL Server Import

1. Right click the database and import wizard - select the server. 
2. Select the CSV file to import - make sure the location is set to the correct database from the default of master.
3. Look at the variable types and allow null values. If these are not correct then the table will not be imported. It will tell you which rows and why so you can alter to float, text, varchar etc.

In this case, the saledate is set as Date and the sale price has been set to text (it contains $ symbols). 

### Stage 2 - Standardise the Date Format

Using convert to get the date into a simple date rather than date/time. An interesting thing to note here for the workplace, is that for tables you have not created it is better to cast as a date and leave the original data how it was... You don't want to update the data at the source without permission.

In [ ]:
Select SaleDate, CONVERT(Date,Saledate)
From Nashville_Housing_Project.dbo.NashvilleHousing

Update NashvilleHousing 
SET SaleDate = CONVERT(Date, Saledate)

### Stage 3 - Populate Property Address Data

Some of the property addresses are left blank i.e. are Null values. Interestingly, the parcel ID is unique to the address so if there is another entry with that parcel ID that does have a property address we can fill any blanks.

In [ ]:
-- Populate Property Address Data
Select *
From Nashville_Housing_Project.dbo.NashvilleHousing
Where PropertyAddress is Null -- you can see the properties that have no property address input

---- you can also see from the data that there is a unique parcel ID for each property address 
---- so if there is a property address associated to the parcel ID you can fill in any property 
---- address blanks for subsequent parcel IDs. 

Select a.ParcelID, a.PropertyAddress, b.ParcelID, b.PropertyAddress, ISNULL(a.PropertyAddress, b.PropertyAddress) 
--use the ISNULL to say if the value is null then replace a.propertyaddress with b.property address
From Nashville_Housing_Project.dbo.NashvilleHousing as a --joining the table to itself using the parcel ID
JOIN Nashville_Housing_Project.dbo.NashvilleHousing as b
    ON a.ParcelID = b.ParcelID
    and a.[uniqueID] <> b.[UniqueID] -- this says where the unique IDs are NOT equal
Where a.PropertyAddress is NULL -- this only cares about the ones where the values in a are null

----To update the table we use update and then set to change to the new values.
update a --using the table alias
SET PropertyAddress = ISNULL(a.PropertyAddress, b.PropertyAddress)
From Nashville_Housing_Project.dbo.NashvilleHousing as a --joining the table to itself using the parcel ID
JOIN Nashville_Housing_Project.dbo.NashvilleHousing as b
    ON a.ParcelID = b.ParcelID
    and a.[uniqueID] <> b.[UniqueID]
Where a.PropertyAddress is NULL

### Stage 4 - Breaking out Address into Individual Columns
(Address, City, State)

The property address column has comma separated values.

__Using substring:__

In [ ]:
SUBSTRING(
 <Column>,
 <Starting position>,
 <End position>
)

-- Character Index
--CHARINDEX can be used to find the position of a character
--Useful in this case to find the comma position to enter as <End position>
--It returns a number representing position of the character in the string

CHARINDEX(
 '<character to look for>',
 <where to look>
)

In [ ]:
Select 
SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress)-1) as address -- the substring here asks for the column, the start and then the end position. 
-- The end position is set using CHARINDEX to locate the comma 
-- so the new substring is from position 1 until this comma i.e. removing anything after. 
-- The -1 then selects everything before the comma and if not included then the new substring will have a comma attached to it.
FROM NashvilleHousing

__Adding new columns and updating the values in the table__

In [ ]:
--Altering table to add in new columns
ALTER TABLE <Table name>
ADD <New Variable Name> <Data type> --Adds new column

--Setting the new column values
UPDATE <Table name>
SET <New Variable Name> = <new variable>

In [ ]:
--create 2 new columns to add values in

---- Add column for Split Address and Split City
ALTER TABLE NashvilleHousing 
ADD PropertySplitAddress NVARCHAR(255) -- adding New column to the end of the table

ALTER TABLE NashvilleHousing
ADD PropertySplitCity NVARCHAR(255)

----Set the values for the new columns
Update NashvilleHousing
SET PropertySplitAddress = SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress)-1) 

UPDATE NashvilleHousing
SET PropertySplitCity = SUBSTRING(PropertyAddress, CHARINDEX(',', PropertyAddress)+1, len(propertyaddress))

__Using PARSENAME to update Owner Address Column__
In this case there are three separated sections. The Street, Town and State. 

Interestingly, the PARSENAME runs backwards so the last piece is 1 (opposite to SUBSTRING).

In [ ]:
--It returns the specified part of the specified object name
PARSENAME (
    <object name>,
    <object piece>
)

PARSENAME looks for '.' and so the commas in the address can be replaced with '.' to allow it to work. To replace commas with full stops REPLACE is used:

In [ ]:
--If the commas were already full stops it would simply be
parsename (OwnerAddress, 3)

--However, they have to be replaced so it becomes
Select
parsename (Replace(OwnerAddress,',', '.') ,3),
parsename (Replace(OwnerAddress,',', '.') ,2),
parsename (Replace(OwnerAddress,',', '.') ,1)
From Nashville_Housing_Project.dbo.NashvilleHousing

___Note: To delete columns (as I had to do after making spelling mistakes):___

In [ ]:
-- Removing columns to rename without spelling error
ALTER TABLE <Table Name> DROP COLUMN <Column Name>, <Column Name>, etc..

Adding the new columns for split owner address:

In [ ]:
----Adding new columns and values
ALTER TABLE NashvilleHousing 
ADD 
OwnerSplitAddress NVARCHAR(255), 
OwnerSplitCity NVARCHAR(255),
OwnerSplitState NVARCHAR(255)

UPDATE NashvilleHousing
SET 
OwnerSplitAddress = parsename (Replace(OwnerAddress,',', '.') ,3),
OwnerSplitCity = parsename (Replace(OwnerAddress,',', '.') ,2),
OwnerSplitState = parsename (Replace(OwnerAddress,',', '.') ,1)

### Stage 5 - Altering Y and N to Yes and No

__Using Cast__: This is used to treat a variable type as an alternative type.

In this instance, my SoldAsVacant column is set to text and so when trying to use DISTINCT, an error occurs. To overcome this, cast can be used:

In [ ]:
CAST(
    <variable> AS <datatype(length)> 
    )

--Note how datatype length is only required for varchar etc.

__To update the Variable type permanently:__ 
_only do this when you are the owner of the table or have permission_

In [ ]:
ALTER TABLE <YourTableHere>
ALTER COLUMN <YourTextColumnHere> VARCHAR(MAX)

__Using CASE to change N to No and Y to Yes__:

In [ ]:
Select SoldAsVacant
, CASE when SoldAsVacant = 'Y' Then 'Yes'
       when SoldAsVacant = 'N' Then 'No'
       Else SoldAsVacant
       END
from Nashville_Housing_Project.dbo.NashvilleHousing

Update NashvilleHousing
Set SoldAsVacant = CASE when SoldAsVacant = 'Y' Then 'Yes'
       when SoldAsVacant = 'N' Then 'No'
       Else SoldAsVacant
       END
    from Nashville_Housing_Project.dbo.NashvilleHousing

Select distinct(SoldAsVacant), count(SoldAsVacant)
From NashvilleHousing
Group By SoldAsVacant
order by 2

### Stage 6 - Deleting Duplicates

In this instance, we are pretending that if the ParcelID, Property Address, Sale Date, Sale Price and Legal Reference are the same then we can treat it as a duplicate. 

_in practice these may be slightly different sales or properties but for this case study these are the parameters used to qualify as a duplicate_ 

Using a CTE to delete duplicate entries - the CTE is set up and then the delete calls everything from that CTE where the row_num is greater than 1 i.e. every duplicate entry...

_I have removed the legal reference as I could not change the variable type and the operation won't work with text type_

In [ ]:
WITH RowNumCTE AS (
Select *,
    ROW_NUMBER() OVER (
        PARTITION BY ParcelID,
                     PropertyAddress,
                     SalePrice,
                     SaleDate
                     ORDER BY 
                        UniqueID
                        ) as row_num
    From Nashville_Housing_Project.dbo.NashvilleHousing
)

DELETE 
From RowNumCTE
Where row_num >1 